## Genetic Algorithm

The problem we will try to solve here is to find the maximum of a 3D function similar to a hat. It is defined as 

$$f(x, y) = sin(sqrt(x^2 + y^2))$$

We will limit our problem to the boundaries of 4 ≥ x ≥ -4 and 4 ≥ y ≥ -4.

The first step is to generate our initial population. A population or generation is our current set of possible solutions, called individuals. We will iterate over several generations improving it until we find an acceptable solution. The first generation is randomly generated.

In [1]:
import random

def generate_population(size, x_boundaries, y_boundaries):
    lower_x_boundary, upper_x_boundary = x_boundaries
    lower_y_boundary, upper_y_boundary = y_boundaries

    population = []
    for i in range(size):
        individual = {
            "x": random.uniform(lower_x_boundary, upper_x_boundary),
            "y": random.uniform(lower_y_boundary, upper_y_boundary),
        }
        population.append(individual)

    return population

Fitness function: Evaluate the fitness of induviduals.
The individuals with the best fitness should be preserved and reproduce while the worst ones should fall — just like in nature. In our case, how we want to find our function maximum, we can simply apply our objective function to an individual and the biggest numbers will be the biggest fitness as well. If we'd want to find the minimum, fitness could be expressed as the result of the function times -1, so smaller values become larger fitness.

In [4]:
import math

def apply_function(individual):
    x = individual["x"]
    y = individual["y"]
    return math.sin(math.sqrt(x ** 2 + y ** 2))

Since we have a population generator and a fitness evaluator, we can start reproducing our individuals
and create the next generation until we find an acceptable solution. 
There are several stop criteria, a largely used 
one is "n generations with stale fitness", but we will use a simpler one,
which is simply n generations - we will use 100. Up to now our entry function looks like:



In [5]:
generations = 100

population = generate_population(size=10, x_boundaries=(-4, 4), y_boundaries=(-4, 4))

i = 1
while True:
    print(f"🧬 GENERATION {i}")

    for individual in population:
        print(individual)

    if i == generations:
        break

    i += 1

🧬 GENERATION 1
{'x': 2.698158018643844, 'y': 0.3985761707258444}
{'x': 2.382882328621845, 'y': 0.1234801231468241}
{'x': -0.8655050025614246, 'y': -1.6964721904036395}
{'x': 2.8098704548377063, 'y': -0.2546958541971174}
{'x': 0.5458232285072411, 'y': 1.3896055174572464}
{'x': 3.9212289995238754, 'y': 2.131946703376501}
{'x': -0.24683642662491945, 'y': -2.1202309409562137}
{'x': -2.121919756386877, 'y': -0.4518700828253843}
{'x': -0.7225578333631919, 'y': -1.5677778112862217}
{'x': 3.0511468167379876, 'y': 3.66571302777999}
🧬 GENERATION 2
{'x': 2.698158018643844, 'y': 0.3985761707258444}
{'x': 2.382882328621845, 'y': 0.1234801231468241}
{'x': -0.8655050025614246, 'y': -1.6964721904036395}
{'x': 2.8098704548377063, 'y': -0.2546958541971174}
{'x': 0.5458232285072411, 'y': 1.3896055174572464}
{'x': 3.9212289995238754, 'y': 2.131946703376501}
{'x': -0.24683642662491945, 'y': -2.1202309409562137}
{'x': -2.121919756386877, 'y': -0.4518700828253843}
{'x': -0.7225578333631919, 'y': -1.567777811

To select the individuals to reproduce we will use a widely adopted method called roulette wheel which consists of dividing a circle in portions like a pie chart, where each individual has a portion proportional to its fitness, and then spinning it. This way we assure best individuals have a better chance of being selected, while the worst ones still have a chance, although it is minor.


Another way to select is the tournament selection. In tournament selection we first choose two individuals at random from our population (it is possible that two low fitness individuals may be chosen). We then pass those individuals to a ‘tournament’ where the individual with the highest fitness will be chosen.

It is possible to further modify this so that the highest fitness individual will win with a given probability.

It is also possible to have more than two individuals in a tournament. The more individuals in a tournament the more the picked population will be biased towards the highest fitness individuals.

Let's say we have four individuals: A, B, C and D with fitness 0, 50, 200 and 250 respectively. The sum of the total fitness is 500, so each one will have a fitness / total_fitness chance of being selected: 0%, 10%, 40%, 50%. We select a random number between 0 and 1 and then verify which individual is in the selected portion: A [0, 0], B (0, 0.1], C (0.1, 0.5], D (0.5, 1].





Since our scenario can have negative fitness, first we have to normalize our individuals by picking the lowest fitness, multiplying by -1 and then adding it to all of them (for example, if we have two individuals with fitness -10 and 5 respectively we add 10 to both becoming 0 and 15). We also expect the population argument to be sorted ascending by fitness so it's easier to get the worst and best individuals.

In [6]:
def choice_by_roulette(sorted_population, fitness_sum):
    offset = 0
    normalized_fitness_sum = fitness_sum

    lowest_fitness = apply_function(sorted_population[0])
    if lowest_fitness < 0:
        offset = -lowest_fitness
        normalized_fitness_sum += offset * len(sorted_population)

    draw = random.uniform(0, 1)

    accumulated = 0
    for individual in sorted_population:
        fitness = apply_function(individual) + offset
        probability = fitness / normalized_fitness_sum
        accumulated += probability

        if draw <= accumulated:
            return individual

Let's then populate the next generation. It should have the same length of the first one, so we will iterate 10 times selecting two individuals each using our roulette and then crossing them. The resultant individual will receive a minor perturbation (mutation) so we don't stick to comfort zone and look out for even better solutions than what we have so far.





There are several crossover techniques for real numbers: for example, we could take x of the individual A and y of the individual B, we could take the geometric mean of each or, the simplest one, take the arithmetic mean of each. If we were dealing with binary data, the most common technique is to pick a part of the bit string of A and a part of bit string of B. For simplicity reasons, let's use the arithmetic mean.





For the mutation there are plenty options too - we will simply sum a small random number between a fixed interval. This interval is the mutation rate and can be fine tuned accordingly, let's use [-0.05, 0.05]. For larger search spaces you can choose larger intervals and diminish it from generation to generation. When dealing with binary data you can simply flip randomly selected bits of the individual string.

In [7]:
def sort_population_by_fitness(population):
    return sorted(population, key=apply_function)


def crossover(individual_a, individual_b):
    xa = individual_a["x"]
    ya = individual_a["y"]

    xb = individual_b["x"]
    yb = individual_b["y"]

    return {"x": (xa + xb) / 2, "y": (ya + yb) / 2}


def mutate(individual):
    next_x = individual["x"] + random.uniform(-0.05, 0.05)
    next_y = individual["y"] + random.uniform(-0.05, 0.05)

    lower_boundary, upper_boundary = (-4, 4)

    # Guarantee we keep inside boundaries
    next_x = min(max(next_x, lower_boundary), upper_boundary)
    next_y = min(max(next_y, lower_boundary), upper_boundary)

    return {"x": next_x, "y": next_y}


def make_next_generation(previous_population):
    next_generation = []
    sorted_by_fitness_population = sort_population_by_fitness(previous_population)
    population_size = len(previous_population)
    fitness_sum = sum(apply_function(individual) for individual in population)

    for i in range(population_size):
        first_choice = choice_by_roulette(sorted_by_fitness_population, fitness_sum)
        second_choice = choice_by_roulette(sorted_by_fitness_population, fitness_sum)

        individual = crossover(first_choice, second_choice)
        individual = mutate(individual)
        next_generation.append(individual)

    return next_generation

In [8]:
generations = 100

population = generate_population(size=10, x_boundaries=(-4, 4), y_boundaries=(-4, 4))

i = 1
while True:
    print(f"🧬 GENERATION {i}")

    for individual in population:
        print(individual, apply_function(individual))

    if i == generations:
        break

    i += 1

    population = make_next_generation(population)

best_individual = sort_population_by_fitness(population)[-1]
print("\n🔬 FINAL RESULT")
print(best_individual, apply_function(best_individual))

🧬 GENERATION 1
{'x': 2.727372334081667, 'y': 2.269443498731267} -0.3953933671842351
{'x': 3.8864256183364283, 'y': -3.782433981154931} -0.7578322146027909
{'x': -1.6991170304419692, 'y': -3.8582309956479053} -0.8792113965765581
{'x': -3.6459696974823554, 'y': -3.8658680468125235} -0.8244549921867352
{'x': -0.6970125568257544, 'y': 3.461064989884524} -0.37922569214413027
{'x': 1.4553624306807107, 'y': 3.16743951659977} -0.33744404254557103
{'x': -1.347934987809479, 'y': -1.7351704176484093} 0.8101333953868761
{'x': 0.9680027870233969, 'y': 3.1633660918078723} -0.16579697176365005
{'x': 1.049948555880242, 'y': 2.1444479137202253} 0.6844913359420003
{'x': 0.5395212718198898, 'y': -3.1679353985581633} -0.07189437880177488
🧬 GENERATION 2
{'x': -0.07125021077299738, 'y': 0.17397707803762957} 0.18689612009052597
{'x': 2.467716419773412, 'y': -0.8127390434991674} 0.5171210390527853
{'x': 1.0602102132146578, 'y': 2.094501955916698} 0.7131938874217489
{'x': 0.7980555385609889, 'y': -0.0465016583